In [170]:
from src.harness import architecture as arch
from src.harness import evolution as evo
from src.harness import utils

import copy
from enum import Enum
import functools
from importlib import reload
import itertools
from matplotlib import pyplot as plt
import multiprocess as mp
import numpy as np
import tensorflow as tf
from tensorflow import keras
from typing import Any, Callable, Dict, Iterable, List, Literal, Set, Tuple

reload(arch)
reload(evo)

<module 'src.harness.evolution' from '/gpfs1/home/j/b/jbourde2/lottery-tickets/src/harness/evolution.py'>

In [171]:
model_features = [
    evo.ModelFeatures.layer_sparsity, 
    evo.ModelFeatures.magnitude,
    evo.ModelFeatures.random,
]
arch_features = [
    evo.ArchFeatures.layer_num,
    evo.ArchFeatures.layer_ohe,
    evo.ArchFeatures.layer_prop_params,
]

individual = evo.Individual("lenet", "mnist", model_features, arch_features, [])

<class 'list'>
[1.0,
 0.064033195,
 0.42257961579401015,
 0.0,
 1.0,
 0.0,
 0.0,
 0.0,
 0.8821874648362777]


In [11]:
num_runs = 5

architecture = arch.Architecture('lenet', 'mnist')
model_features = [
    evo.Features.layer_sparsity, 
    evo.Features.magnitude,
    evo.Features.random,
]
arch_features = [
    evo.ArchFeatures.layer_num,
    evo.ArchFeatures.layer_ohe,
    evo.ArchFeatures.layer_prop_params,
]
layers = [(4, 'relu')]

individual_constructor = functools.partial(
    evo.Individual, 
    architecture=architecture,
    model_features=model_features,
    arch_features=arch_features,
    layers=layers,
)

objectives = [
    (evo.Target.MAXIMIZE, lambda x: 1, evo.Individual.eval_accuracy),
    (evo.Target.MINIMIZE, lambda x: 1, evo.Individual.sparsity),
]

rate_func = lambda n: 0.5
scale_func = lambda n: 1 / np.sqrt(n + 1)
mutations = [
    functools.partial(evo.Individual.get_annealing_mutate(), rate=rate_func, scale=scale_func)
]
    
genome_metric_callbacks = [
    evo.Population.average_sparsity
]
kwargs = {
    "num_generations": 2,
    "archive_size": 10,
    "population_size": 10,
    "fronts_to_consider": 2,
    "tournament_size": 4,
    "num_tournament_winners": 2,
    "individual_constructor": individual_constructor,
    "objectives": objectives,
    "mutations": mutations,
    "crossover": evo.Individual.crossover,
    "genome_metric_callbacks": genome_metric_callbacks,
}

all_genome_metrics = []
all_objective_metrics = []
all_archives = []
for run in range(num_runs):
    print(f"Run {run + 1}")
    genome_metrics, objective_metrics, archive = evo.nsga2(**kwargs)
    all_genome_metrics.append(genome_metrics)
    all_objective_metrics.append(objective_metrics)
    all_archives.append(archive)

Run 1
Generation 1


I0000 00:00:1731597992.527764   49363 service.cc:146] XLA service 0x2b5fe00026b0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1731597992.527910   49363 service.cc:154]   StreamExecutor device (0): Tesla V100-SXM2-32GB, Compute Capability 7.0
2024-11-14 10:26:32.560267: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-11-14 10:26:32.676797: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8906
I0000 00:00:1731597993.591124   49363 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


Selection with 2 fronts and 2 sparsities


/users/j/b/jbourde2/.conda/envs/lt/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:713: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 6 variables whereas the saved optimizer has 2 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


Generation 2
Selection with 2 fronts and 2 sparsities


KeyboardInterrupt: 